# 프로젝트: 멋진 작사가 만들기

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import tensorflow

### Step 2. 데이터 읽어오기

In [22]:
import glob
import os, re
import numpy as np
import tensorflow as tf

txt_file_path = '/content/drive/MyDrive/data (1)/*'
11
 

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation']


# 데이터 정제

In [23]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Looking for some education
Made my way into the night
All that bullshit conversation
Baby, can't you read the signs? I won't bore you with the details, baby
I don't even wanna waste your time
Let's just say that maybe
You could help me ease my mind
I ain't Mr. Right But if you're looking for fast love
If that's love in your eyes
It's more than enough


텍스트를 인공지능 모델에 학습시키기 위해서는 특수 문자를 제거해야 한다.

In [24]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 모든 것을 소문자로 바꾸고, 양쪽 공백을 지운다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 특수문자 양쪽에 공백을 넣는다.
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러 개의 공백은 하나의 공백으로 바꾼다.
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾼다
    sentence = sentence.strip() # 다시 양쪽 공백을 지운다
    sentence = '<start> ' + sentence + ' <end>' # 문장 시작에는 <start>, 끝에는 <end>를 추가
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


단어만을 다루기 때문에 특수문자를 없앴다. 그리고, 문장의 시작과 끝을 알려줬다.

In [25]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    corpus.append(preprocess_sentence(sentence))
        
# 10개를 확인
corpus[:10]

['<start> looking for some education <end>',
 '<start> made my way into the night <end>',
 '<start> all that bullshit conversation <end>',
 '<start> baby , can t you read the signs ? i won t bore you with the details , baby <end>',
 '<start> i don t even wanna waste your time <end>',
 '<start> let s just say that maybe <end>',
 '<start> you could help me ease my mind <end>',
 '<start> i ain t mr . right but if you re looking for fast love <end>',
 '<start> if that s love in your eyes <end>',
 '<start> it s more than enough <end>']

특수문자가 제거된 것을 확인할 수 있다.

In [26]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다

def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>" # 사전에 없었던 단어를 사용할 수 있게 토큰으로 대체
    )
    tokenizer.fit_on_texts(corpus) # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tensor = tokenizer.texts_to_sequences(corpus) # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다   

    total_data_text = list(tensor)
    num_tokens = [len(tokens) for tokens in total_data_text]
    max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
    maxlen = int(max_tokens)
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=maxlen)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 304  28 ...   0   0   0]
 [  2 221  13 ...   0   0   0]
 [  2  24  17 ...   0   0   0]
 ...
 [  2 100   6 ...   0   0   0]
 [  2 122   9 ...   0   0   0]
 [  2  74   6 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f3c9bc62490>


데이터를 전처리 해줘야 한다. 문장을 단어 단위로 나누어 컴퓨터가 이해할 수 있도록 변환시켜야 한다.

단, 평균 길이보다 큰 outlier가 있을 수 있는 데, 이는 제거해줘야 한다. 컴퓨터가 읽기 힘들어 한다. 이를 해결하기 위해 maxlen을 잘 설정해야 한다.

In [27]:
# 값이 잘 나오는 지 확인

print(tensor[:3, :10])

[[   2  304   28   99 4811    3    0    0    0    0]
 [   2  221   13   85  226    6  115    3    0    0]
 [   2   24   17 1087 2347    3    0    0    0    0]]


In [28]:
# 토큰으로 잘 쪼개어져 있는 지 확인

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [29]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
tgt_input = tensor[:, 1:]  # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.  

print(src_input[0])
print(tgt_input[0])

[   2  304   28   99 4811    3    0    0    0    0    0    0    0    0
    0    0    0    0    0]
[ 304   28   99 4811    3    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]


길이 조절이 잘 되어 있는 것을 확인할 수 있다.

# 평가 데이터셋 분리

In [30]:
# 단어장 크기 12000이상, 총 데이터의 20%

from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True, 
                                                          random_state=34)


In [31]:
# 행열 잘 나왔는 지 확인

print('Source Train: ', enc_train.shape)
print('Target Train: ', dec_train.shape)

Source Train:  (140599, 19)
Target Train:  (140599, 19)


# 인공지능 만들기

In [32]:
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [33]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = Embedding(vocab_size, embedding_size)
        self.rnn_1 = LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = LSTM(hidden_size, return_sequences=True)
        self.linear = Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 19
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

# 모델 학습

In [34]:
history = []
epochs = 10

optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)

history = model.fit(enc_train, 
          dec_train, 
          epochs=epochs,
          batch_size=256,
          validation_data=(enc_val, dec_val),
          verbose=1)

Epoch 1/10
550/550 [==============================] - 173s 286ms/step - loss: 2.8043 - val_loss: 2.4827
Epoch 2/10
550/550 [==============================] - 157s 286ms/step - loss: 2.3915 - val_loss: 2.3263
Epoch 3/10
550/550 [==============================] - 165s 300ms/step - loss: 2.2255 - val_loss: 2.2082
Epoch 4/10
550/550 [==============================] - 157s 286ms/step - loss: 2.0568 - val_loss: 2.0963
Epoch 5/10
550/550 [==============================] - 165s 300ms/step - loss: 1.8750 - val_loss: 2.0052
Epoch 6/10
550/550 [==============================] - 157s 286ms/step - loss: 1.6867 - val_loss: 1.9348
Epoch 7/10
550/550 [==============================] - 157s 286ms/step - loss: 1.5057 - val_loss: 1.8812
Epoch 8/10
550/550 [==============================] - 157s 286ms/step - loss: 1.3439 - val_loss: 1.8546
Epoch 9/10
550/550 [==============================] - 165s 300ms/step - loss: 1.2069 - val_loss: 1.8357
Epoch 10/10
550/550 [==============================] - 165s 300m

# 가사 만들기

In [35]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나 씩 분류해놨다. 그래서 하나씩 입력하여 문장을 만들어야 한다.
    while True:

        # 입력받은 문장의 텐서를 입력합니다.
        predict = model(test_tensor)  
        # 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 2에서 예측된 word index를 문장 뒤에 붙입니다 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated 

In [36]:
# 가사 한 줄 만들어 내보자 !

generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you so much <end> '

# 회고


많은 분들의 도움으로  원하는 loss 값을 쉽게 잘 찾은 것 같다. 이번 ex는 다른 것들과 다르게 train, test를 나누지 않고 train 안 에서 스스로 test를 나눠 모델링 했다는 것이 신기했다. 처음 시작할 때, 신경망이 사람을 본 따서 만들었다는 것을 의심했다. 인간을 본따서 구현한다는 것이 이질적이었다. 하지만, 코드를 직접 하나씩 써보면서 메커니즘 흐름을 따라가니 정교하게 잘만들어졌다는 생각이 들었다.

아이펠에서는 2가지 과정이 있다. 자연어 처리와 영상처리.. 그 중 하나를 경험했다는 것이 고무적이다. 다음에 있을 영상처리가 기대된다.

출처

https://github.com/PEBpung/Aiffel/blob/master/Project/Exploration/E11.%20%EC%9E%91%EC%82%AC%EA%B0%80%20%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5%20%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb